In [3]:
import os
import sys
# sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

In [4]:
out_path = "../corpus/novel_formated/"
corpus_name = "novel_segments.tsv"


In [5]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [6]:
len(conv_data)

1045266

In [7]:
conv_data_mini = conv_data[:len(conv_data)//100]

In [8]:
len(conv_data_mini)

10452

In [10]:
def make_Src_Tgt(conv_data):
    src_str = []
    tgt_str = []

    for conv in conv_data:
        src_str.append(conv[-2])
        tgt_str.append(conv[-1])
    return src_str, tgt_str

In [11]:
src_str, tgt_str = make_Src_Tgt(conv_data_mini)

In [12]:
# from gensim.models import KeyedVectors
# w2v_path = "../corpus/w2v/"
# w2v_name =  "model.vec"
# w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

In [20]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(texts):
    
    counter = Counter()
    for text in tqdm(texts):
        counter.update( [t.orth_ for t in nlp(text)] )
    return Vocab(counter, specials=['<pad>', '<unk>','<fos>', '<eos>', '<del>'])

In [23]:
vocab_src = build_vocab(src_str[:500])
vocab_tgt = build_vocab(tgt_str[:500])

100%|██████████| 500/500 [00:05<00:00, 88.29it/s]


In [22]:
for char, index in list(vocab_src.stoi.items())[:15]:
    print('Char: {: <8} → Index: {: <2}'.format(char, index))

Char: <pad>    → Index: 0 
Char: <unk>    → Index: 1 
Char: <fos>    → Index: 2 
Char: <eos>    → Index: 3 
Char: <del>    → Index: 4 
Char: 、        → Index: 5 
Char: !        → Index: 6 
Char: 。        → Index: 7 
Char: …        → Index: 8 
Char: は        → Index: 9 
Char: だ        → Index: 10
Char: の        → Index: 11
Char: ?        → Index: 12
Char: て        → Index: 13
Char: に        → Index: 14


In [24]:
PAD_IDX = vocab_src.stoi["<pad>"]

0

In [ ]:
def data_process(texts_src, texts_tgt, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt):
    
    data = []
    for (src, tgt) in zip(texts_src, texts_tgt):
        src_tensor = torch.tensor(
            convert_text_to_indexes(text=src, vocab=vocab_src, tokenizer=tokenizer_src),
            dtype=torch.long
        )
        tgt_tensor = torch.tensor(
            convert_text_to_indexes(text=tgt, vocab=vocab_tgt, tokenizer=tokenizer_tgt),
            dtype=torch.long
        )
        data.append((src_tensor, tgt_tensor))
        
    return data

def convert_text_to_indexes(text, vocab):
    return [vocab['<fos>']] + [
        vocab[token.orth_] for token in nlp(text)
    ] + [vocab['<eos>']]

In [ ]:
from sklearn.model_selection import train_test_split
import random

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    
    def forward(self, sequence):
        embedding = self.word_embeddings(sequence)
        # Many to Oneなので、第２戻り値を使う
        _, state = self.lstm(embedding)
        # state = (h, c)
        return state